# Tensors

In [1]:
import torch

In [2]:
# 生成一个tensors
x = torch.Tensor(5, 3)
print(x)


1.00000e-29 *
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  3.9873
[torch.FloatTensor of size 5x3]



In [3]:
# 随机生成一个tensors
x = torch.rand(5, 3)
print(x)


 0.6439  0.1664  0.2390
 0.1366  0.1889  0.4560
 0.3873  0.0750  0.9990
 0.6260  0.9631  0.9666
 0.5728  0.6236  0.8652
[torch.FloatTensor of size 5x3]



In [4]:
# 获取size
print(x.size())

torch.Size([5, 3])


# Operations

In [5]:
# 加法
y = torch.rand(5, 3)
x + y


 1.4217  0.9937  0.2933
 0.5797  0.7853  1.2179
 0.9804  0.4841  1.8811
 1.6093  1.8934  1.8078
 1.4445  1.1813  0.8987
[torch.FloatTensor of size 5x3]

In [6]:
# 加法
torch.add(x, y)


 1.4217  0.9937  0.2933
 0.5797  0.7853  1.2179
 0.9804  0.4841  1.8811
 1.6093  1.8934  1.8078
 1.4445  1.1813  0.8987
[torch.FloatTensor of size 5x3]

In [7]:
# 加法，获取结果
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
result


 1.4217  0.9937  0.2933
 0.5797  0.7853  1.2179
 0.9804  0.4841  1.8811
 1.6093  1.8934  1.8078
 1.4445  1.1813  0.8987
[torch.FloatTensor of size 5x3]

In [8]:
# 加法，在原tensor上相加
y.add_(x)
y


 1.4217  0.9937  0.2933
 0.5797  0.7853  1.2179
 0.9804  0.4841  1.8811
 1.6093  1.8934  1.8078
 1.4445  1.1813  0.8987
[torch.FloatTensor of size 5x3]

In [9]:
# 索引
x[:, 1]


 0.1664
 0.1889
 0.0750
 0.9631
 0.6236
[torch.FloatTensor of size 5]

# Numpy Bridge
torch tensor和 numpy array共享内存

In [10]:
# 将torch Tensor 转换成为 numpy Array
a = torch.ones(5)
b = a.numpy()
b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [11]:
# 将numpy array 转换成为torch Tensor
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[ 2.  2.  2.  2.  2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]



# 自动求导

In [12]:
from torch.autograd import Variable

In [13]:
# 创建一个变量
x = Variable(torch.ones(2, 2), requires_grad=True)
x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [14]:
# 进行一个操作
y = x + 2
y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [15]:
# 进行更多操作
z = y * y * 3
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



In [16]:
# 求导
out.backward()
x.grad

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

# Neural Network

In [17]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 一个图像信道，6个输入信道，5X5个核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 全连接层
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        """前向传播
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [19]:
# 返回网络的参数
params = list(net.parameters())
print(len(params))
print(params[0].size()) # 返回conv1's weight

10
torch.Size([6, 1, 5, 5])


In [20]:
# 输入是Variable变量
# torch.nn支持批量的样本，不支持单个样本，如nn.Conv2d支持nSample x nChannel x Height x Width
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
out

Variable containing:
 0.0352  0.0344 -0.0944  0.0105  0.0745 -0.0232  0.0385  0.1048 -0.1283 -0.0024
[torch.FloatTensor of size 1x10]

In [21]:
# 将所有的梯度设置为0，并且采用随机梯度回调
net.zero_grad()
out.backward(torch.randn(1, 10))

# Loss Function

In [22]:
output = net(input)
target = Variable(torch.arange(1, 10))
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

/Users/qisc/.pyenv/versions/miniconda3-4.1.11/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.3. Note that arange generates values in [start; end), not [start; end].
  from ipykernel import kernelapp as app


Variable containing:
 38.4996
[torch.FloatTensor of size 1]

# Backprop

In [23]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()

print(net.conv1.bias.grad)

Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

Variable containing:
 0.0290
 0.1181
 0.0408
 0.0002
-0.0331
 0.0854
[torch.FloatTensor of size 6]



# 更新权重

In [25]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [26]:
# 更新权重的方法有SGD, Nesterov-SGD, Adam, RMSProp等等。
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 更新
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 更新